# Quickstart: Vector search using Gemini Embeddings and Elasticsearch

This tutorial demonstrates how to use the [Gemini API](https://ai.google.dev/docs) to create [embeddings](https://ai.google.dev/docs/embeddings_guide) and store them in Elasticsearch. Elasticsearch will enable us to perform vector search (Knn) to find similar documents.

## setup

* Elastic Credentials - Create [Cloud deployment](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud) to get all Elastic credentials (`ELASTIC_CLOUD_ID`, `ELASTIC_API_KEY`).

* `GOOGLE_API_KEY` - To use the Gemini API, you need an API key. [Follow](https://ai.google.dev/tutorials/setup) to create a key with one click in Google AI Studio.

## Install packages

In [ ]:
pip install -q -U google-generativeai elasticsearch

## Import packages and credentials

In [ ]:
import google.generativeai as genai
import google.ai.generativelanguage as glm
from elasticsearch import Elasticsearch, helpers
from getpass import getpass

GOOGLE_API_KEY = getpass("Google API Key :")
ELASTIC_API_KEY = getpass("Elastic API Key :")
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID :")
elastic_index_name = "gemini-demo"

## Embedding generation



In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)

title = "Climate in India"
sample_text = "India generally experiences a hot summer from March to June, with temperatures often exceeding 40°C in central and northern regions. Monsoon season, from June to September, brings heavy rainfall, especially in the western coast and northeastern areas. Post-monsoon months, October and November, mark a transition with decreasing rainfall. Winter, from December to February, varies in temperature across the country, with colder conditions in the north and milder weather in the south. India's diverse climate is influenced by its geographical features, resulting in regional "

model = "models/embedding-001"
embedding = genai.embed_content(
    model=model, content=sample_text, task_type="retrieval_document", title=title
)

## Connecting Elasticsearch

In [ ]:
es = Elasticsearch(cloud_id=ELASTIC_CLOUD_ID, api_key=ELASTIC_API_KEY)

## Index document with Elasticsearch

In [ ]:
doc = {"text": sample_text, "text_embedding": embedding["embedding"]}

resp = es.index(index=elastic_index_name, document=doc)

print(resp)

## Searching for document with Elasticsearch

In [ ]:
q = "How's weather in India?"

embedding = genai.embed_content(model=model, content=q, task_type="retrieval_query")

resp = es.search(
    index=elastic_index_name,
    knn={
        "field": "text_embedding",
        "query_vector": embedding["embedding"],
        "k": 10,
        "num_candidates": 100,
    },
)


for result in resp["hits"]["hits"]:
    pretty_output = f"\n\nID: {result['_id']}\n\nText: {result['_source']['text']}\n\nEmbedding: {result['_source']['text_embedding']}"
    print(pretty_output)